<a href="https://colab.research.google.com/github/VishalPrem1994/AIGenPlayGround/blob/main/SD_Img2Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai numpy opencv-python accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers diffusers omegaconf requests

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [99]:
import requests
import torch
from PIL import Image
from io import BytesIO
import torch
from torch import autocast
from diffusers import StableDiffusionImg2ImgPipeline
import os

model_path = "/content/drive/MyDrive/stable_diffusion_weights/RevAnim"
torch.cuda.empty_cache()

device = "cuda"
model_id_or_path = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16)
pipe = pipe.to(device)

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [175]:
selected_folder = "Bike1" #@param {type:"string"}
prompt = "anime, highly detailed, intricate, natural lighting" #@param {type:"string"}
negative_prompt = "(worst quality:1.4), (low quality:1.4), child, deformed, deformed face, bad hands, bad fingers, bad hands, (3d, render, cgi, doll, painting:1.4), long body, blurry, duplicate, cloned, duplicate body parts, disfigured, extra limbs, fused fingers, extra fingers, twisted, malformed hands, mutated hands and fingers, conjoined, missing limbs, bad anatomy, bad proportions, logo, signature, text, words, lowres, boring, mutated, artifacts, gross, ugly, stretch, smooth skin texture"
num_samples = 1
guidance_scale = 8 #@param {type:"number"}
strength = 0.6 #@param {type:"number"}
num_inference_steps = 50

In [ ]:
import os
main_path = "/content/drive/MyDrive/AI/VideoInput/"
fullpath = main_path+selected_folder
print(fullpath)
output_folder = fullpath+"/Output"
os.makedirs(output_folder, exist_ok=True)

with autocast("cuda"), torch.inference_mode():
  for i in os.listdir(fullpath):
    print(i)
    if i.endswith(".jpg"):
      num = i.split("-")[1]
      print(num)
      init_image = Image.open(fullpath+"/"+i).convert("RGB")
      images = pipe(
          prompt=prompt,
          image=init_image,
          strength=strength,
          guidance_scale=guidance_scale,
          num_images_per_prompt = num_samples,
          num_inference_steps = num_inference_steps,
          generator=g_cuda
          ).images
      output_path = fullpath+"/"+"Output/"+"image2-"+str(num)
      print(output_path)
      images[0].save(output_path)

In [167]:
import cv2
import os

image_folder = output_folder
print(output_folder)
video_name = output_folder+'/video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
images.sort()
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 5, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

/content/drive/MyDrive/AI/VideoInput/Bike2/Output
